In [ ]:
library(data.table)
library(dplyr)
library(readxl)
library(ggplot2)
library(purrr)
library(stringr)
library(sva)
library(limma)
library(edgeR)
library(DESeq2)
library(vsn)
library(hexbin)
library(FactoMineR)
library(factoextra)
library(variancePartition)
library(tidyr)
library(pheatmap)
library(EnhancedVolcano)

In [ ]:
#Guid limma-trend
#https://bioconductor.org/packages/devel/bioc/vignettes/limma/inst/doc/usersguide.pdf
# The use of eBayes or treat with trend=TRUE is known as the limma-trend method (Law et al, 2014;
# Phipson et al, 2016). With this option, an intensity-dependent trend is fitted to the prior variances
# s2.prior. Specifically, squeezeVar is called with the covariate equal to Amean, the average
# log2-intensity for each gene. The trend that is fitted can be examined by plotSA. limma-trend is
# useful for processing expression values that show a mean-variance relationship. This is often useful
# for microarray data, and it can also be applied to RNA-seq counts that have been converted to
# log2-counts per million (logCPM) values (Law et al, 2014). When applied to RNA-seq logCPM
# values, limma-trend give similar results to the voom method. The voom method incorporates the
# mean-variance trend into the precision weights, whereas limma-trend incorporates the trend into
# the empirical Bayes moderation. limma-trend is somewhat simpler than voom because it assumes
# that the sequencing depths (library sizes) are not wildly different between the samples and it applies
# the mean-variance trend on a genewise basis instead to individual observations. limma-trend is
# recommended for RNA-seq analysis when the library sizes are reasonably consistent (less than
# 3-fold difference from smallest to largest) because of its simplicity and speed.

In [ ]:
# These samples are filtered for duplicates, life-time diagnosis, and new cases samples from the new rna-seq run
samples.info = read.table("/grehawi/splice-reg-prj/new-data/Diff-Analysis/combined_pheno_withCT.csv")
dim(samples.info)

In [ ]:
#set preparation_pbmc_batch for IST samples to a new value 33
samples.info$preparation_pbmc_batch[samples.info$sequence_run == 2] = 33

In [ ]:
head(samples.info)

In [ ]:
#Read gene count (not normalised, not batch corrected)
genes.count = read.table("/grehawi/splice-reg-prj/new-data/RNA-seq/gene_counts_combined_noDex.matrix")
head(genes.count)
dim(genes.count)

In [ ]:
hist(as.matrix(genes.count))

In [ ]:
# Form a data frame analogous to expression data that will hold the samples info.
genes.count = genes.count[, colnames(genes.count) %in% samples.info$combined_id]
dim(genes.count)
samples = colnames(genes.count)
order = match(samples, samples.info$combined_id)
samples.info = samples.info[order, ]
rownames(samples.info) = samples.info$combined_id

head(samples.info)
dim(samples.info)

## 1. Explore possible covariates (we know already that we have the following confounders with huge effect) including: seq_run , mean_gc_content, total_pairs

In [ ]:
# seq-run effect
#svg("/grehawi/splice-reg-prj/pca-seqrun.svg", width = 10, height = 8)
options(repr.plot.width=12, repr.plot.height=8)
pca_genes <- PCA(
  X = t(scale(genes.count)),
      ncp = 10,
  graph = FALSE)
#variance PC1(52.8%), PC2(11.6%)
fviz_pca_ind(pca_genes, col.ind="Sequence_run",
             geom = "point", labelsize= 40,
            habillage = as.factor(samples.info$sequence_run)) + theme_classic() +
            theme(axis.text= element_text(size = 17), axis.title.x = element_text(size = 20),
                    axis.title.y = element_text(size = 20), legend.text=element_text(size=22),
                    legend.title=element_text(size=22)) + labs(x = "PC1 (52.8%)", y = "PC2 (11.6%)")
#dev.off()

In [ ]:
#Sex effect
fviz_pca_ind(pca_genes, 
             geom = "point", labelsize= 30,
            habillage = as.factor(samples.info$sex))

In [ ]:
# Barplot of sex distribution
options(repr.plot.width=8, repr.plot.height=6)
first_run_sex_distr = samples.info[samples.info$sequence_run == 1, colnames(samples.info) %in% c('sex')]
second_run_sex_distr = samples.info[samples.info$sequence_run == 2, colnames(samples.info) %in% c('sex')]
table(first_run_sex_distr)
table(second_run_sex_distr)
Sex = samples.info[, colnames(samples.info) %in% c('sex')]

ggplot(as.data.frame(table(Sex)), aes(x=Sex, y=Freq)) + 
geom_bar(stat = "identity", fill=c("#EBC30C", "yellowgreen"))+ theme_classic() +
theme(axis.text.x = element_text(size = 17), axis.title.x = element_text(size = 20),
        axis.text.y = element_text(size = 17), axis.title.y = element_text(size = 20)) + labs(y = "Count")
ggsave("/grehawi/splice-reg-prj/barplot_sex_distr.png", width = 12, height = 6)

In [ ]:
#BMI effect
#svg("/grehawi/splice-reg-prj/barplot_BMI_distr.svg", width = 10, height = 8)
# Create a blank plot
hist(samples.info$BMI[samples.info$ltany_di == 1],
     main = "Histogram of BMI Values",
     xlab = "BMI",
     col = "#F0CF7F",
     border = "black", ylab = 'Count', cex.lab = 1.5, cex.axis = 1.3)

# Add another histogram
hist(samples.info$BMI[samples.info$ltany_di == 0], add = TRUE, col = "#5DB9B5", border = "black") 

# Create a legend
legend("topright", legend = c("Affected Individuals", "Unaffected Individuals"),
       fill = c("#F0CF7F", "#5DB9B5"), bty = 'n', cex = 1.5)
#dev.off()

In [ ]:
# Age effect
options(repr.plot.width=8, repr.plot.height=6)
#svg("/grehawi/splice-reg-prj/barplot_age_distr.svg", width = 10, height = 8)
# Create a blank plot
hist(samples.info$age[samples.info$ltany_di == 1],
     main = "Histogram of Age Values",
     xlab = "Age",
     col = "#F0CF7F",
     border = "black", ylab = 'Count', cex.lab = 1.5, cex.axis = 1.3)

# Add another histogram
hist(samples.info$age[samples.info$ltany_di == 0], add = TRUE, col = "#5DB9B5", border = "black")

# Create a legend
legend("topright", legend = c("Affected Individuals", "Unaffected Individuals"),
       fill = c("#F0CF7F", "#5DB9B5"), bty = 'n', cex = 1.5)
#dev.off()

# Cases are older than controls, we should add this in our design matrix

In [ ]:
#read cell type info
ct = read.table("/grehawi/splice-reg-prj/new-data/batch_corr/dtangle-cellTypes-on-raw-LM22.txt")
head(ct)
dim(ct)
ct = ct[rownames(ct) %in% samples.info$combined_id, ]
dim(ct)
# Get x number of PCs to represent cell-type values
ct_scaled = scale(ct)
pca_cellT <- PCA(
  X = ct_scaled)
pca_cellT$eig
pcs_cellT <- pca_cellT$ind$coord
pcs_cellT
# The first 5 pcs explain 54% of variance in the data

## 2. Perform gene-level Batch correction

In [ ]:
# perform limma-trend for DEA
design <- model.matrix(~ ltany_di, data = samples.info)
design
dge <- DGEList(counts=genes.count)
dge <- calcNormFactors(dge)
logCPM <- cpm(dge, log=TRUE, prior.count=3)

In [ ]:
hist(logCPM)

### Correct for seq-run as it's the largest covariate correcting for more covariates

In [ ]:
covars <- c("ltany_di")
batches <- c("sequence_run")
design <- model.matrix(~sequence_run + ltany_di , 
                       data=samples.info[, colnames(samples.info) %in% c(covars, batches)])
head(design)
dim(design)

In [ ]:
treatment.design <- design[,c(1,3)]
batch.design = design[,c(2)]

gene.count.seqrun.corrected <- removeBatchEffect(logCPM,covariates=batch.design, design=treatment.design)

In [ ]:
# genes with negative values after correction
with.negative.values = gene.count.seqrun.corrected[apply(gene.count.seqrun.corrected[, ], 1, min) < 0, ]
dim(with.negative.values)
gene.count.noNeg = logCPM[!row.names(logCPM) %in% row.names(with.negative.values), ]
head(gene.count.noNeg)
dim(gene.count.noNeg)

In [ ]:
gene.count.seqrun.corrected.final <- removeBatchEffect(gene.count.noNeg,covariates=batch.design, design=treatment.design)

### SVA
The idea is to generate surrogate variables that capture unexplained variance in the data. You can specify biological variables whose variance should not be included in the surrogate variables. Here, we only specify the case/control status (as determined by the ltany_di==any diagnosis data). We use the data corrected for sequence_run

In [ ]:
model_biol <- model.matrix(~as.factor(ltany_di), data = samples.info)
model_0 <- model.matrix(~1, data = samples.info)
sva_res <- svaseq(
  dat = as.matrix(gene.count.seqrun.corrected.final),
  mod = model_biol,
  mod0 = model_0,
)

### Canonical correlation of possible confounders

In [ ]:
pca_gene.count.sr.corrected <- PCA(
  X = t(gene.count.seqrun.corrected.final),
  ncp = 10,
  graph = FALSE)

In [ ]:
#svg("/grehawi/splice-reg-prj/heatmap_correl_confounders.svg", width = 12, height = 12)

#add Cell types PCs to sample info
colnames(pcs_cellT) <- paste0("cellType_pc_", seq_len(ncol(pcs_cellT)))
samplesInfo_combined_with_cellTypes <- cbind(samples.info, as.data.frame(pcs_cellT))

#add SVs to sample info
sva_svs <- sva_res$sv
colnames(sva_svs) <- paste0("sv_", seq_len(ncol(sva_svs)))
samplesInfo_combined_with_cellTypes_svs <- cbind(samplesInfo_combined_with_cellTypes, as.data.frame(sva_svs))

# add PCs to sample info
pca_pc <- pca_gene.count.sr.corrected$ind$coord
colnames(pca_pc) <- paste0("pc_", seq_len(ncol(pca_pc)))
samplesInfo_combined_with_cellTypes_svs_pcs <- cbind(samplesInfo_combined_with_cellTypes_svs, as.data.frame(pca_pc))


#Define biological and technical covariates
covars <- c("sex", "age", "BMI", "study", "ltany_di")
batches <- c("plates", "RIN", "mean_gc_content", "total_pairs", "preparation_pbmc_batch", 
             paste0("sv_", seq_len(ncol(sva_svs))), paste0("cellType_pc_", seq_len(ncol(pcs_cellT))))
vars <- c(covars, batches, paste0("cellType_pc_", seq_len(ncol(pcs_cellT))),paste0("pc_", seq_len(ncol(pca_pc))))

# this is the correlation of the meta-variables with each other
form_c <- as.formula(paste0("~", paste0(vars, collapse = " + ")))
cancor_res <- canCorPairs(form_c, samplesInfo_combined_with_cellTypes_svs_pcs)

options(repr.plot.width=12, repr.plot.height=12)
plotCorrMatrix(cancor_res, margins=c(20,20), cex.lab = 2)
#dev.off()

In [ ]:
# pc1, sv1,pc2, sv2 mean_gc, total_pairs, RIN are correlated
# total_pairs and celltype_pc2 are highly correlated 
# study is correlated with status and celltype_pc2 and total_pairs
# plates and prep_batch are correlated and are both corr to mean_gc_content
#RIN and mean_gc_contetn are highly correlated so correcting for one of them is enough (Previously investigated)

### Calculate association of confounders & PCs

In [ ]:
# for every PC, calculate the association (for every covariate)
# The p-value for R^2 comes from the measure F = ss(mean) - ss(fit) / ss(fit)
covars <- c("sex", "age", "BMI", "ltany_di")
batches <- c("plates", "RIN", "mean_gc_content", "total_pairs", "preparation_pbmc_batch",
             paste0("sv_", seq_len(ncol(sva_svs[, 1:5]))), paste0("cellType_pc_", seq_len(ncol(pcs_cellT))))
vars <- c(covars, batches,paste0("pc_", seq_len(ncol(pca_pc))))

association_results <- set_names(paste0("pc_", 1:10)) %>% 
  map_dfr(function(pc) {
    # for every covariate
    map_dfr(c(covars, batches), function(var) {
      res <- anova(lm(samplesInfo_combined_with_cellTypes_svs_pcs[, pc] ~ samplesInfo_combined_with_cellTypes_svs_pcs[, var]))
      data.frame(pc = pc, var = var, p_val = res$`Pr(>F)`[1])
    })
  })

col <- c("gray",
         colorRampPalette(rev(c("#D73027", "#FC8D59", "#FEE090", "#FFFFBF",
                                "#E0F3F8", "#91BFDB", "#4575B4")))(100))

association_results_wide <- association_results %>% 
  pivot_wider(
    names_from = "var",
    values_from = "p_val"
  )

# exclude the pc column
association_results_wide_matrix <- as.matrix(association_results_wide[, -1])
rownames(association_results_wide_matrix) <- association_results_wide$pc

final_matrix = -log10(t(association_results_wide_matrix))
# sv1 is highly correlated with pc1 that the result of -log10 is inf -->fix
final_matrix[10, 1] <- 220

options(repr.plot.width=11, repr.plot.height=8)
pheatmap(final_matrix, color = col, fontsize=14,
         cluster_cols = FALSE)
gt <- pheatmap(final_matrix, color = col, fontsize=14,
         cluster_cols = FALSE, width=11, hight=6, filename="heatmap_correl_confounders_pcs.pdf")$gtable
ggsave("/grehawi/splice-reg-prj/heatmap_correl_confounders_pcs.pdf", plot = gt)

### Correct for batch effects and biological covariates

In [ ]:
# perform limma-trend for DEA
design <- model.matrix(~ ltany_di + pcs_cellT + sex + age + BMI + mean_gc_content + total_pairs, data = samples.info)
design

In [ ]:
expr.design <- design[,c(1,2)]
batch.design = design[,c(3:12)]

gene.count.corrected <- removeBatchEffect(gene.count.seqrun.corrected.final,covariates=batch.design, design=expr.design)

In [ ]:
# genes with negative values after correction
with.negative.values = gene.count.corrected[apply(gene.count.corrected[, ], 1, min) < 0, ]
dim(with.negative.values)
gene.count.noNeg = gene.count.seqrun.corrected.final[!row.names(gene.count.seqrun.corrected.final) %in% row.names(with.negative.values), ]
head(gene.count.noNeg)
dim(gene.count.noNeg)

In [ ]:
# Redo batch correction after removing genes which appear to have negative values after correction
gene.count.corrected.final <- removeBatchEffect(gene.count.noNeg,covariates=batch.design, design=expr.design)

In [ ]:
write.table(gene.count.corrected.final, "/grehawi/splice-reg-prj/new-data/Diff-Analysis/gene.logCPM.corrected.final.matrix")

## 3. Perform transcript-level Batch correction

In [ ]:
#Read the batch corrected trx counts
trx.counts = read.table("/grehawi/splice-reg-prj/new-data/RNA-seq/trx_counts_combined_noDex.matrix")
trx.counts = trx.counts[, colnames(trx.counts) %in% samples.info$combined_id]
head(trx.counts)
dim(trx.counts)

In [ ]:
# perform limma-trend for DEA
design <- model.matrix(~ ltany_di, data = samples.info)
design
dge <- DGEList(counts=trx.counts)
dge <- calcNormFactors(dge)
trx.logCPM <- cpm(dge, log=TRUE, prior.count=3)

### Correct for seq-run

In [ ]:
covars <- c("ltany_di")
batches <- c("sequence_run")
design <- model.matrix(~sequence_run + ltany_di , 
                       data=samples.info[, colnames(samples.info) %in% c(covars, batches)])

treatment.design <- design[,c(1,3)]
batch.design = design[,c(2)]

trx.count.seqrun.corrected <- removeBatchEffect(trx.logCPM,covariates=batch.design, design=treatment.design)

In [ ]:
# trx with negative values after correction
with.negative.values = trx.count.seqrun.corrected[apply(trx.count.seqrun.corrected[, ], 1, min) < 0, ]
dim(with.negative.values)
trx.count.noNeg = trx.logCPM[!row.names(trx.logCPM) %in% row.names(with.negative.values), ]
head(trx.count.noNeg)
dim(trx.count.noNeg)

In [ ]:
trx.count.seqrun.corrected.final <- removeBatchEffect(trx.count.noNeg,covariates=batch.design, design=treatment.design)

### Correct for batch effect and biological covaraites

In [ ]:
# perform limma-trend for DEA
design <- model.matrix(~ ltany_di + pcs_cellT + sex + age + BMI + mean_gc_content + total_pairs, data = samples.info)
design


In [ ]:
expr.design <- design[,c(1,2)]
batch.design = design[,c(3:12)]

trx.count.corrected <- removeBatchEffect(trx.count.seqrun.corrected.final,covariates=batch.design, design=expr.design)

In [ ]:
# trx with negative values after correction
with.negative.values = trx.count.corrected[apply(trx.count.corrected[, ], 1, min) < 0, ]
dim(with.negative.values)
trx.count.noNeg = trx.count.seqrun.corrected.final[!row.names(trx.count.seqrun.corrected.final) %in% row.names(with.negative.values), ]
head(trx.count.noNeg)
dim(trx.count.noNeg)

In [ ]:
# Redo batch correction after removing genes which appear to have negative values after correction
trx.count.corrected.final <- removeBatchEffect(trx.count.noNeg,covariates=batch.design, design=expr.design)

In [ ]:
write.table(trx.count.corrected.final, "/grehawi/splice-reg-prj/new-data/Diff-Analysis/trx.logCPM.corrected.final.matrix")